## VGG16 pretrained model and train

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
from torchvision.models import vgg16

# Set Seed for Reproducibility
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

# Dataset Paths- Update with your paths
train_dir = '/Users/samuel/CS5640/FinalProjectPt2/state-farm-distracted-driver-detection/imgs/train'
best_model_path = "best_vgg_model.pth"

# Data Augmentation and Preprocessing
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.RandomResizedCrop((224, 224), scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load Training Dataset
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)

# Split Training Dataset
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# Data Loaders
batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

# Initialize Pretrained VGG Model
model = vgg16(pretrained=True)
model.classifier[6] = nn.Linear(4096, len(train_dataset.dataset.classes))
model = model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# Loss, Optimizer, and Scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)

# Training Function
def train_vgg(
    model,
    train_loader,
    val_loader,
    criterion,
    optimizer,
    scheduler,
    device,
    epochs=20,
    patience=5
):
    scaler = torch.cuda.amp.GradScaler()  # For mixed precision training
    best_val_loss = float('inf')
    early_stop_counter = 0

    for epoch in range(epochs):
        # Training Phase
        model.train()
        running_train_loss = 0.0
        train_progress = tqdm(
            enumerate(train_loader),
            total=len(train_loader),
            desc=f"Epoch {epoch + 1}/{epochs} Training",
            unit="batch"
        )

        for batch_idx, (inputs, labels) in train_progress:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            with torch.cuda.amp.autocast():  # Enable mixed precision
                outputs = model(inputs)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            running_train_loss += loss.item()
            train_progress.set_postfix(loss=f"{loss.item():.4f}")

        avg_train_loss = running_train_loss / len(train_loader)

        # Validation Phase
        model.eval()
        running_val_loss = 0.0
        correct, total = 0, 0
        all_labels, all_preds = [], []

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                running_val_loss += loss.item()

                _, preds = torch.max(outputs, 1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
                all_labels.extend(labels.cpu().numpy())
                all_preds.extend(preds.cpu().numpy())

        avg_val_loss = running_val_loss / len(val_loader)
        val_accuracy = correct / total

        print(f"\nEpoch {epoch + 1}/{epochs}: "
              f"Train Loss: {avg_train_loss:.4f}, "
              f"Val Loss: {avg_val_loss:.4f}, "
              f"Val Accuracy: {val_accuracy:.4f}")

        # Save the Best Model
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            early_stop_counter = 0
            torch.save(model.state_dict(), best_model_path)
            print(f"Saved Best Model at Epoch {epoch + 1}")
        else:
            early_stop_counter += 1
            if early_stop_counter >= patience:
                print(f"Early Stopping at Epoch {epoch + 1}")
                break

        # Update Learning Rate
        scheduler.step()

    # Load the Best Model for Final Evaluation
    model.load_state_dict(torch.load(best_model_path))
    print("Best Model Loaded.")

    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_preds)
    print("\nConfusion Matrix:\n", cm)


# Train the Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_vgg(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    device=device,
    epochs=20,
    patience=5
)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /Users/samuel/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [01:35<00:00, 5.80MB/s] 
/var/folders/8q/w3n8_d9j4p74p399mm4pdcbh0000gn/T/ipykernel_76294/788730090.py:70: FutureWarning: `torch.cuda.amp.Gra


Epoch 1/20: Train Loss: 0.6997, Val Loss: 0.1319, Val Accuracy: 0.9652
Saved Best Model at Epoch 1


Epoch 2/20 Training: 100%|██████████| 141/141 [1:17:46<00:00, 33.10s/batch, loss=0.2452]



Epoch 2/20: Train Loss: 0.0930, Val Loss: 0.0846, Val Accuracy: 0.9732
Saved Best Model at Epoch 2


Epoch 3/20 Training: 100%|██████████| 141/141 [1:18:26<00:00, 33.38s/batch, loss=0.0652]


...


Epoch 12/20: Train Loss: 0.0021, Val Loss: 0.0201, Val Accuracy: 0.9955


Epoch 13/20 Training: 100%|██████████| 141/141 [1:20:26<00:00, 34.23s/batch, loss=0.0001]



Epoch 13/20: Train Loss: 0.0031, Val Loss: 0.0212, Val Accuracy: 0.9960


Epoch 14/20 Training:  77%|███████▋  | 109/141 [1:23:30<24:31, 45.97s/batch, loss=0.0012]  


KeyboardInterrupt: 

## create submission on test dataset

In [ ]:
import os
import csv
import torch
from torchvision import transforms, models
from PIL import Image
from tqdm import tqdm
import logging

# Configure Logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Dataset Paths- Update with your paths
test_dir = '/Users/samuel/CS5640/FinalProjectPt2/state-farm-distracted-driver-detection/imgs/test'
best_model_path = "/Users/samuel/CS5640/SamuelBlakeFinal/distracted-driving-behaviors/Custom_VGG/best_vgg_model.pth"
submission_file = "/Users/samuel/CS5640/FinalProjectPt2/state-farm-distracted-driver-detection/Custom_VGG/vgg_submission.csv"

# Hyperparameters
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 64
NUM_CLASSES = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logging.info(f"Using device: {device}")

# Transformations
transform_val_test = transforms.Compose([
    transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # ImageNet normalization
])

# VGG Model
def get_vgg_model(num_classes):
    model = models.vgg16(pretrained=False)
    # Adjust the final fully connected layer to match the number of classes
    model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, num_classes)
    return model

# Function to Process a Batch of Images
def process_batch(image_paths):
    images = []
    img_names = []
    for img_path in image_paths:
        img_name = os.path.basename(img_path)
        img = Image.open(img_path).convert("RGB")
        img = transform_val_test(img)
        images.append(img)
        img_names.append(img_name)

    images = torch.stack(images).to(device)
    return images, img_names

# Ensure Directory Exists for Saving Model
directory = os.path.dirname(best_model_path)
if not os.path.exists(directory):
    os.makedirs(directory)
    logging.info(f"Directory {directory} created.")

# Check if Model File Exists Before Loading
if os.path.exists(best_model_path):
    logging.info("Loading the best VGG model...")
    best_model = get_vgg_model(NUM_CLASSES).to(device)
    best_model.load_state_dict(torch.load(best_model_path, map_location=device))
    best_model.eval()
    logging.info("Model loaded successfully.")
else:
    logging.error(f"Model file {best_model_path} not found. Please train and save the model first.")
    raise FileNotFoundError(f"{best_model_path} not found.")

# Load Test Data
logging.info("Reading test images...")
test_image_paths = [os.path.join(test_dir, img) for img in os.listdir(test_dir) if img.lower().endswith(('.jpg', '.jpeg', '.png'))]
if len(test_image_paths) == 0:
    raise ValueError("No valid image files found in the test directory!")
logging.info(f"Found {len(test_image_paths)} test images.")

# Generate Predictions and Save to CSV
logging.info("Generating predictions...")
fieldnames = ["img"] + [f"c{i}" for i in range(NUM_CLASSES)]

with open(submission_file, mode="w", newline="") as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()

    for i in tqdm(range(0, len(test_image_paths), BATCH_SIZE), desc="Processing Batches", unit="batch"):
        batch_paths = test_image_paths[i:i + BATCH_SIZE]
        images, img_names = process_batch(batch_paths)

        with torch.no_grad():
            outputs = best_model(images)
            probabilities = torch.softmax(outputs, dim=1).cpu().numpy()

        for j, img_name in enumerate(img_names):
            row = {"img": img_name}
            row.update({f"c{k}": probabilities[j][k] for k in range(NUM_CLASSES)})
            writer.writerow(row)

logging.info(f"Submission file '{submission_file}' created successfully.")

2024-12-03 19:29:29,279 - INFO - Using device: cpu
2024-12-03 19:29:29,280 - INFO - Loading the best VGG model...
/var/folders/8q/w3n8_d9j4p74p399mm4pdcbh0000gn/T/ipykernel_77639/1039228071.py:63: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an i